In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import collections
import numpy as np
import pandas as pd

In [3]:
from IPython.display import Image
import tensorflow as tf

In [4]:
import os
import urllib

In [5]:
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

In [6]:
IRIS_TRAINING = "./data/iris_training.csv"
IRIS_TEST = "./data/iris_test.csv"

INPUT_TENSOR_NAME = 'inputs'

model_path = "./model"
model_path_serving = "./model_serving"

### Estimator Defintion DNN

In [8]:
def estimator(model_path):
    feature_columns = [tf.feature_column.numeric_column(INPUT_TENSOR_NAME, shape=[4])]
    return tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                      hidden_units=[10, 20, 30],
                                      n_classes=3,
                                      model_dir=model_path
                                    )

### Define Input for test and Train

In [11]:
def train_input_fn():
    training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
        filename=IRIS_TRAINING,
        target_dtype=np.int,
        features_dtype=np.float32)

    return tf.estimator.inputs.numpy_input_fn(
        x={INPUT_TENSOR_NAME: np.array(training_set.data)},
        y=np.array(training_set.target),
        num_epochs=None,
        shuffle=True)()
def test_input_fn():
    testing_set = tf.contrib.learn.datasets.base.load_csv_with_header(
        filename=IRIS_TEST,
        target_dtype=np.int,
        features_dtype=np.float32)

    return tf.estimator.inputs.numpy_input_fn(
        x={INPUT_TENSOR_NAME: np.array(testing_set.data)},
        y=np.array(testing_set.target),
        num_epochs=1,
        shuffle=False)()

In [13]:
def serving_input_receiver_fn():
    feature_spec = {INPUT_TENSOR_NAME: tf.FixedLenFeature(dtype=tf.float32, shape=[4])}
    return tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)()

In [14]:
classifier = estimator(model_path)

# Train model using the train input function, for 2000 steps
classifier.train(input_fn=train_input_fn, steps=2000)

# Evaluate accuracy of the trained model using the test input pipeline
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_model_dir': './model', '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_session_config': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./model\model.ckpt.
INFO:tensorflow:loss = 299.317, step = 1
INFO:tensorflow:global_step/sec: 265.982
INFO:tensorflow:loss = 11.1377, step = 101 (0.380 sec)
INFO:tensorflow:global_step/sec: 308.642
INFO:tensorflow:loss = 16.4644, step = 201 (0.276 sec)
INFO:tensorflow:global_step/sec: 333.332
INFO:tensorflow:loss = 12.4134, step = 301 (0.376 sec)
INFO:tensorflow:global_step/sec: 274.725
INFO:tensorflow:loss = 6.02933, step = 401 (0.336 sec)
INFO:tensorflow:global_step/sec: 316.456
INFO:tensorflow:loss = 1.82696, step = 501 (0.316 sec)
INFO:tensorflow:global_step/sec: 294.1

## Export Trained Model

In [ ]:
save_path = classifier.export_savedmodel(model_path_serving, serving_input_receiver_fn)

INFO:tensorflow:Restoring parameters from ./model\model.ckpt-2000


In [ ]:
# Create methods that would maintain the session using the model path as an id
sess_dict = {}
def get_session(model_id):
    global sess_dict
    config = tf.ConfigProto(allow_soft_placement=True)
    sess_dict[model_id] = tf.Session(config=config)
    return sess_dict[model_id]

def load_tf_model(model_path):
    sess = get_session(model_path)
    tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], model_path)
    return sess

In [ ]:
sess = load_tf_model(save_path)

In [ ]:
# Use the restored graph's get_operations() method to know all the names of the ops available in the current graph
for op in tf.get_default_graph().get_operations():
    print (str(op.name))

In [ ]:
input_x_holder = sess.graph.get_operation_by_name("input_example_tensor").outputs[0]
predictions_holder = sess.graph.get_operation_by_name("dnn/head/predictions/probabilities").outputs[0]

In [ ]:
# Predict on a new set of inputs [6.7, 2.5, 5.8, 1.8] which belongs to class 2
float_features = tf.train.Feature(float_list=tf.train.FloatList(value=[6.7, 2.5, 5.8, 1.8]))
feature_dict = {"inputs": float_features}
example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
serialized = example.SerializeToString()
score = sess.run([predictions_holder], {input_x_holder: [serialized]})

In [ ]:
print(["%.4f" % score[0][0][idx] for idx in range(len(score[0][0]))])

In [ ]:
# Let us print the index of the largest score 
print(np.argmax(score))